# PoC: Run AA0.5 Scenario - Band 2 MID Imaging
Based on https://confluence.skatelescope.org/display/SST/AA0.5+Scenarios?preview=/145374457/145376035/AA0.5_MID_Scenario_Imaging_in_Band_2.docx

## <font color='green'>Notebook set-up</font>

### Import Python Modules and Dummy Commands
The commands in omc_dummy_commands are currently dummy commands which don't match the reality of OMC / OET / TMC commands.

In [ ]:
import json
import time
import numpy as np
from bokeh.plotting import figure, output_file, save
from bokeh.io import show, output_notebook, push_notebook
from bokeh.layouts import row
from bokeh.models import LabelSet, ColumnDataSource
output_notebook()
from omc_dummy_commands import *

### Fixed parameters (for this PoC)

In [ ]:
int_time = 5.0 #-- time in seconds for correlator integration time
speed_up = 500.0 #-- totally artificial speed up factor for time
num_chans = 512 #-- number of channels 
total_time = 0.0
off_colour = 'darkmagenta'
on_colour = 'forestgreen' 
on_angle = 1.57/2.0
off_angle = 0.0

## <font color='green'>General Monitoring Setup </font>

Doesn't yet exist... but some hints might be here: https://notebooks.gesis.org/binder/jupyter/user/bokeh-bokeh-notebooks-zlebh904/notebooks/tutorial/11%20-%20Running%20Bokeh%20Applications.ipynb

In [ ]:
#--- 'plot' for Simple monitoring --#
monitor_plot = figure(plot_width=900, plot_height=400,toolbar_location=None,
                      x_range=(-0.5, 1.5), y_range=(-0.5, 1.5))
monitor_plot.axis.visible = False  # -- Hide axes
monitor_plot.grid.visible = False  # -- Hide grid

#-- initial paramters -#

monitor_source = ColumnDataSource(dict(label_x = [0.5, 0.5, 0.5], label_y = [0.0, 0.5, 1.0], 
                                       label_text = ['On/Off', 'Resources Assigned?', 'Other thing'], 
                                       label_angle = [off_angle, off_angle, off_angle],
                                       label_color = [off_colour, off_colour, off_colour]))

labels = LabelSet(x='label_x', y='label_y', text='label_text', 
        x_offset=-200, y_offset=-10, render_mode='canvas',text_color='black', source=monitor_source)

monitor_plot.square(x = 'label_x',y = 'label_y', size = 20, color = 'label_color', 
                    angle='label_angle', source=monitor_source)

monitor_plot.add_layout(labels)

In [ ]:
#-- Amplitude vs time/chan plots --#
amp_time = figure(plot_width=450, plot_height=400,toolbar_location=None,
                  x_axis_label = "Time [s]", y_axis_label = 'Amplitude')
amp_avg = figure(plot_width=450, plot_height=400,toolbar_location=None, 
                 x_axis_label = "Channels", y_axis_label = 'Amplitude')
amp_time.line(0,0)
amp_avg.line(0,0)


In [ ]:
monitor_frame = show(monitor_plot, notebook_handle=True)
amp_plots = show(row(amp_time,amp_avg), notebook_handle=True)


## <font color='green'>Begin observation sequence</font> 

### Telescope On/Off

In [ ]:
telescope_onoff('on', monitor_frame, monitor_source, on_angle, on_colour, off_angle, off_colour)

### Assign Resource

Reads *CentralNode.AssignResources* from https://confluence.skatelescope.org/display/SWSI/Configuration+Schemas into a ...

In [ ]:
resources = assign_resources('cn_assignres_mid.json', monitor_frame, monitor_source, on_angle,
                             on_colour, off_angle, off_colour)
show_resources(resources, resource_type="")

### Configure the system

Load desired observational state from a JSON string.

Configurations covers CSP (CBF), DISH, SPFRx, SDP.



In [ ]:
#TBD

### Set-up a sub array

At AA0.5 there will be only 1, but useful to have.

In [ ]:
# TBD

### Start Observing 

Here targets are loaded from stand-alone JSON files (THIS NEEDS CHANGING TO BE MORE SB LIKE).


In [ ]:
cal_obs = observe_loop('simple_cals.json', num_chans, int_time, amp_avg, amp_time,
                       amp_plots, total_time, num_loops = 1)

In [ ]:
targ_obs = observe_loop('simple_targs.json', num_chans, int_time, amp_avg, amp_time,
                        amp_plots, cal_obs[0], num_loops =  2)